# This code generate a Flask Web APP to predict and classify images, detecting if there's Violence or not

This codes includes function to create thumbnails of the videos in the dataset.

It uses the models generated on the Train_violence_model.ipynb file

In [ ]:
# Install necesary libs
!pip install Flask
!pip install imgaug
!pip install pytube

In [ ]:
# Used once for creating thumbnails of videos
import os
import subprocess

# Specify the directory containing your videos
video_directory = './archive/Real Life Violence Dataset/NonViolence'

# Specify the directory where you want to save the thumbnails
thumbnail_directory = './archive/Real Life Violence Dataset/NonViolence/thumbnails'

# Ensure the thumbnail directory exists
if not os.path.exists(thumbnail_directory):
    os.makedirs(thumbnail_directory)

# Iterate over each file in the video directory
for filename in os.listdir(video_directory):
    # Check if the file is a video (you might need to adjust the condition based on your video formats)
    if filename.endswith(('.mp4', '.avi', '.mkv')):
        # Construct the full path to the video file
        video_path = os.path.join(video_directory, filename)
        
        # Construct the output path for the thumbnail
        thumbnail_path = os.path.join(thumbnail_directory, filename + '.jpg')
        
        # Use FFmpeg to generate a thumbnail from the video
        # This command extracts a frame at 10 seconds into the video and saves it as a JPEG
        subprocess.call(['ffmpeg', '-ss', '00:00:1.000', '-i', video_path, '-vframes', '1', thumbnail_path])

print("Thumbnails generated successfully.")

In [1]:
# Function to Extract frames from videos
import cv2
import os
import imageio
import imgaug.augmenters as iaa
import imgaug as ia

IMG_SIZE = 128
ColorChannels = 3

def video_to_frames(video):
    vidcap = cv2.VideoCapture(video)

    import math
    rate = math.floor(vidcap.get(3))
    count = 0

    ImageFrames = []
    while vidcap.isOpened():
        ID = vidcap.get(1)
        success, image = vidcap.read()

        if success:
            # skipping frames to avoid duplications
            if (ID % 7 == 0):
                rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                resized = cv2.resize(rgb_img, (IMG_SIZE, IMG_SIZE))
                ImageFrames.append(resized)

            count += 1
        else:
            break

    vidcap.release()

    return ImageFrames

In [2]:
#Load trained model
from keras.models import load_model

model = load_model('model.h5')

In [3]:
def avg_max(video_name,confidence):
    filen = video_name.split("\\",2)[1]
    v_nv = filen.split("_")[0]
    if v_nv == "NV":
        return np.average(confidence)
    elif v_nv == "V":
        return max(confidence)

def with_moviepy(filename):
    from moviepy.editor import VideoFileClip
    clip = VideoFileClip(filename)
    duration       = clip.duration
    return duration

def newest(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

def record(out):
    global rec_frame
    while(rec):
        time.sleep(0.05)
        out.write(rec_frame)

def gen_frames():  # generate frame by frame from camera
    global out, capture,rec_frame
    while True:
        success, frame = camera.read() 
        if success:
            if(capture):
                capture=0
                now = datetime.datetime.now()
                p = os.path.sep.join(['shots', "shot_{}.png".format(str(now).replace(":",''))])
                cv2.imwrite(p, frame)
            
            if(rec):
                rec_frame=frame
                frame= cv2.putText(cv2.flip(frame,1),"Recording...", (0,25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),4)
                frame=cv2.flip(frame,1)
            
            try:
                ret, buffer = cv2.imencode('.jpg', cv2.flip(frame,1))
                frame = buffer.tobytes()
                yield (b'--frame\r\n'
                       b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            except Exception as e:
                pass
                
        else:
            pass

In [5]:
from flask import Flask, render_template, request, Response, redirect, url_for
import os
import random
from PIL import Image
from keras.preprocessing import image
import numpy as np
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from pytube import YouTube
from pytube.innertube import _default_clients
import random
import datetime, time
import os, sys
from threading import Thread
global capture,rec_frame,rec, out 
capture=0
rec=0

_default_clients["ANDROID_MUSIC"] = _default_clients["ANDROID_CREATOR"]

thumbnails = os.path.join('static','thumbnails')
videos = os.path.join('static')
template_dir = os.path.abspath('./templates')
app = Flask(__name__, template_folder=template_dir)
app.config['UPLOAD_FOLDER'] = thumbnails


@app.route('/')
def list_videos():
    #List random file from directory
    video_directory = './static/'
    all_files = os.listdir(video_directory)
    video_files = [f for f in all_files if f.endswith(('.mp4', '.avi', '.mkv'))]
    random_videos = random.sample(video_files, min(1, len(video_files)))
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], random_videos[0] + '.jpg')
    full_filename_video = os.path.join(videos, random_videos[0])

    all_frames = len(video_to_frames(full_filename_video))
    result = []
    for i in range(all_frames):
        
        image_file = video_to_frames(full_filename_video)[i]
        im = Image.fromarray(image_file)
        im.save("inference.jpeg")
        
        img_width, img_height = 128, 128
        img = image.load_img("inference.jpeg", target_size = (img_width, img_height))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        result.append(model.predict(img))
        
    return render_template('videos.html', videos=random_videos, images = full_filename, detection = avg_max(full_filename_video,result))

@app.route('/select_video', methods=['POST'])
def select_video():
    #List random file from directory
    video_directory = './static'
    all_files = os.listdir(video_directory)
    video_files = [f for f in all_files if f.endswith(('.mp4', '.avi', '.mkv'))]
    random_videos = random.sample(video_files, min(1, len(video_files)))
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], random_videos[0] + '.jpg')
    full_filename_video = os.path.join(videos, random_videos[0])

    all_frames = len(video_to_frames(full_filename_video))
    result = []
    for i in range(all_frames):
        
        image_file = video_to_frames(full_filename_video)[i]
        im = Image.fromarray(image_file)
        im.save("inference.jpeg")
        
        img_width, img_height = 128, 128
        img = image.load_img("inference.jpeg", target_size = (img_width, img_height))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        result.append(model.predict(img))
        
    return render_template('videos.html', videos=random_videos, images = full_filename, detection = avg_max(full_filename_video,result))

@app.route('/Youtube_form', methods=['GET', 'POST'])
def Youtube_form():
    if request.method == 'POST':
        yt = YouTube(request.form['link'], use_oauth=True, allow_oauth_cache=True)
        new_video = yt.streams.first().download('')
        ytvideo = new_video.split("\\",5)[5]
        ffmpeg_extract_subclip(ytvideo, np.int32(request.form['time']), np.int32(request.form['time'])+5, targetname="video.mp4")
        all_frames = len(video_to_frames("video.mp4"))
        result = []
        for i in range(all_frames):
            
            image_file = video_to_frames("video.mp4")[i]
            im = Image.fromarray(image_file)
            im.save("./static/inference.jpeg")
            
            img_width, img_height = 128, 128
            img = image.load_img("./static/inference.jpeg", target_size = (img_width, img_height))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis = 0)
            result.append(model.predict(img))

        return render_template('Youtube_results.html', videos=ytvideo, images = "./inference.jpeg", detection = np.average(result))
     
    return render_template('Youtube_form.html')

@app.route('/Youtube_train', methods=['GET', 'POST'])
def Youtube_train():
    if request.method == 'POST':
        video_type = ''.join(list(request.form.listvalues())[1])
        filename = request.form['video_name']
        dest = os.path.join("./retrain",video_type,filename)
        os.rename(os.path.join("./",filename), dest)
                             
        return render_template('Youtube_train.html', path = dest)

@app.route('/streaming',methods=['GET', 'POST'])
def tasks():
    global switch,camera,result
    camera = cv2.VideoCapture(0)
    if request.method == 'POST':
        if request.form.get('click') == 'Capture':
            global capture
            capture=1
            img_width, img_height = 128, 128
            img = image.load_img(newest("./shots"), target_size = (img_width, img_height))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis = 0)
            result = model.predict(img)
            
        elif  request.form.get('rec') == 'Start/Stop Recording':
            global rec, out, rec_frame
            rec= not rec
            if(rec):
                now=datetime.datetime.now() 
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter('./recordings/vid_{}.avi'.format(str(now).replace(":",'')), fourcc, 20.0, (640, 480))
                #Start new thread for recording the video
                thread = Thread(target = record, args=[out,])
                thread.start()
            elif(rec==False):
                out.release()
                all_frames = len(video_to_frames(newest("./recordings")))
                result = []
                for i in range(all_frames):
                    
                    image_file = video_to_frames(newest("./recordings"))[i]
                    im = Image.fromarray(image_file)
                    im.save("./static/inference.jpeg")
                    
                    img_width, img_height = 128, 128
                    img = image.load_img("./static/inference.jpeg", target_size = (img_width, img_height))
                    img = image.img_to_array(img)
                    img = np.expand_dims(img, axis = 0)
                    result.append(model.predict(img))
        
        return render_template('streaming.html', detection = np.average(result))
    return render_template('streaming.html')
    
@app.route('/video_feed')
def video_feed():
    camera = cv2.VideoCapture(0)
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/rtsp_streaming',methods=['GET', 'POST'])
def tasks_rtsp():
    global switch,camera,result
    # Other feeds https://g33ktricks.blogspot.com/p/the-rtsp-real-time-streaming-protocol.html
    camera = cv2.VideoCapture('http://webcam01.ecn.purdue.edu/mjpg/video.mjpg')
    if request.method == 'POST':
        if request.form.get('click') == 'Capture':
            global capture
            capture=1
            img_width, img_height = 128, 128
            img = image.load_img(newest("./shots"), target_size = (img_width, img_height))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis = 0)
            result = model.predict(img)
            
        elif  request.form.get('rec') == 'Start/Stop Recording':
            global rec, out, rec_frame
            rec= not rec
            if(rec):
                now=datetime.datetime.now() 
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter('./recordings/vid_{}.avi'.format(str(now).replace(":",'')), fourcc, 20.0, (640, 480))
                #Start new thread for recording the video
                thread = Thread(target = record, args=[out,])
                thread.start()
            elif(rec==False):
                out.release()
                all_frames = len(video_to_frames(newest("./recordings")))
                result = []
                for i in range(all_frames):
                    
                    image_file = video_to_frames(newest("./recordings"))[i]
                    im = Image.fromarray(image_file)
                    im.save("./static/inference.jpeg")
                    
                    img_width, img_height = 128, 128
                    img = image.load_img("./static/inference.jpeg", target_size = (img_width, img_height))
                    img = image.img_to_array(img)
                    img = np.expand_dims(img, axis = 0)
                    result.append(model.predict(img))
        
        return render_template('streaming_rtsp.html', detection = np.average(result))
    return render_template('streaming_rtsp.html')
    
@app.route('/video_feed_rtsp')
def video_feed_rtsp():
    # Other feeds https://g33ktricks.blogspot.com/p/the-rtsp-real-time-streaming-protocol.html
    camera = cv2.VideoCapture('http://webcam01.ecn.purdue.edu/mjpg/video.mjpg')
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2024 03:20:14] "GET /Youtube_form HTTP/1.1" 200 -


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1/1 [==============================] - 0s 28ms/step


127.0.0.1 - - [23/Apr/2024 03:20:29] "POST /Youtube_form HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2024 03:20:29] "GET /static/inference.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2024 03:20:32] "POST /Youtube_train HTTP/1.1" 200 -


Non Violence
Held his own He wins a 5 vs 1 fight and gets ovation 6th Street Austin TX.mp4


127.0.0.1 - - [23/Apr/2024 03:20:40] "GET /Youtube_form HTTP/1.1" 200 -


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
1/1 [==============================] - 0s 26ms/step


127.0.0.1 - - [23/Apr/2024 03:20:50] "POST /Youtube_form HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2024 03:20:50] "GET /static/inference.jpeg HTTP/1.1" 200 -
[2024-04-23 03:20:57,405] ERROR in app: Exception on /Youtube_train [POST]
Traceback (most recent call last):
  File "C:\Users\jpti\anaconda3\Lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jpti\anaconda3\Lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jpti\anaconda3\Lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jpti\anaconda3\Lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Violence
Held his own He wins a 5 vs 1 fight and gets ovation 6th Street Austin TX.mp4
